In [ ]:
!pip install langchain openai tiktoken sentence-transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import userdata
HF_Token = userdata.get('HF_Phi2')

print(f"HF Token Loaded")

HF Token Loaded


# Generating Embeddings for RAG

In [4]:
import pandas as pd
import json

# Load Processed Data
combined_data = pd.read_csv('/content/drive/MyDrive/GenAI-CSA/data/combined/combined_data.csv')
with open('/content/drive/MyDrive/GenAI-CSA/data/metadata/metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"Loaded combined_data.csv with {combined_data.shape[0]} rows.")
print(f"Loaded metadata.json with {len(metadata)} entries.")

Loaded combined_data.csv with 501606 rows.
Loaded metadata.json with 501606 entries.


In [5]:
# Integrate Metadata with Text Data
documents = [
    {"page_content": content, "metadata": metadata}
    for content, metadata in zip(combined_data['Data_Summary'], metadata)
]

print(f"Reconstructed {len(documents)} documents for embedding generation.")

Reconstructed 501606 documents for embedding generation.


In [6]:
from sentence_transformers import SentenceTransformer
import pickle
import time

# Load Embedding Generation Model (paraphrase-mpnet-base-v2)
start_time = time.time()
model = SentenceTransformer('paraphrase-mpnet-base-v2')

print(f"Model loaded in {time.time() - start_time:.2f} seconds.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded in 15.35 seconds.


In [7]:
# Prepare Textual Data for Embeddings
texts = [doc['page_content'] for doc in documents]

# Generate Embeddings
start_time = time.time()
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"Embeddings generated in {time.time() - start_time:.2f} seconds.")

Batches:   0%|          | 0/7838 [00:00<?, ?it/s]

Embeddings generated in 2712.62 seconds.


In [11]:
# Save Embeddings
with open('embeddings_mpnet.pkl', 'wb') as f:
    pickle.dump({'documents': documents, 'embeddings': embeddings}, f)

print("Embeddings saved successfully as 'embeddings_mpnet.pkl'")


Embeddings saved successfully as 'embeddings_mpnet.pkl'


In [12]:
# Download Embeddings
from google.colab import files
files.download('embeddings_mpnet.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>